In [1]:
import numpy as np
import os
import sys
import math
from astropy.io import fits
from tqdm import tqdm
from joblib import Parallel, delayed

# ----- class for snapshot header ----- 

class snapshot_header:
    def __init__(self, filename):
        if (not os.path.exists(filename)):
            print("file not found:", filename)
            sys.exit()
      
        self.filename = filename  
        f = open(filename,'rb')    
        blocksize = np.fromfile(f,dtype=np.int32,count=1)
        if blocksize[0] == 8:
            swap = 0
            format = 2
        elif blocksize[0] == 256:
            swap = 0
            format = 1  
        else:
            blocksize.byteswap(True)
            if blocksize[0] == 8:
                swap = 1
                format = 2
            elif blocksize[0] == 256:
                swap = 1
                format = 1
            else:
                print("incorrect file format encountered when reading header of", filename)
                sys.exit()
    
        self.format = format
        self.swap = swap
    
        if format==2:
            f.seek(16, os.SEEK_CUR)
    
        self.npart = np.fromfile(f,dtype=np.int32,count=6)
        self.massarr = np.fromfile(f,dtype=np.float64,count=6)
        self.time = (np.fromfile(f,dtype=np.float64,count=1))[0]
        self.redshift = (np.fromfile(f,dtype=np.float64,count=1))[0]
        self.sfr = (np.fromfile(f,dtype=np.int32,count=1))[0]
        self.feedback = (np.fromfile(f,dtype=np.int32,count=1))[0]
        self.nall = np.fromfile(f,dtype=np.int32,count=6)
        self.cooling = (np.fromfile(f,dtype=np.int32,count=1))[0]
        self.filenum = (np.fromfile(f,dtype=np.int32,count=1))[0]
        self.boxsize = (np.fromfile(f,dtype=np.float64,count=1))[0]
        self.omega_m = (np.fromfile(f,dtype=np.float64,count=1))[0]
        self.omega_l = (np.fromfile(f,dtype=np.float64,count=1))[0]
        self.hubble = (np.fromfile(f,dtype=np.float64,count=1))[0]
    
        if swap:
            self.npart.byteswap(True)
            self.massarr.byteswap(True)
            self.time = self.time.byteswap()
            self.redshift = self.redshift.byteswap()
            self.sfr = self.sfr.byteswap()
            self.feedback = self.feedback.byteswap()
            self.nall.byteswap(True)
            self.cooling = self.cooling.byteswap()
            self.filenum = self.filenum.byteswap()
            self.boxsize = self.boxsize.byteswap()
            self.omega_m = self.omega_m.byteswap()
            self.omega_l = self.omega_l.byteswap()
            self.hubble = self.hubble.byteswap()
     
        f.close()
 


#----- find offset and size of data block ----- 

def find_block(filename, format, swap, block, block_num, only_list_blocks=False):
    if (not os.path.exists(filename)):
        print("file not found:", filename)
        sys.exit()
            
    f = open(filename,'rb')
    f.seek(0, os.SEEK_END)
    filesize = f.tell()
    f.seek(0, os.SEEK_SET)
  
    found = False
    curblock_num = 1
    while ((not found) and (f.tell()<filesize)):
        if format==2:
            f.seek(4, os.SEEK_CUR)
            curblock = f.read(4)
            if (block == curblock):
                found = True
            f.seek(8, os.SEEK_CUR)  
        else:
            if curblock_num==block_num:
                found = True
        
        curblocksize = (np.fromfile(f,dtype=np.int32,count=1))[0]
        if swap:
            curblocksize = curblocksize.byteswap()
    
    # - print some debug info about found data blocks -
    #if format==2:
    #  print curblock, curblock_num, curblocksize
    #else:
    #  print curblock_num, curblocksize
    
        if only_list_blocks:
            print(curblock_num,curblock,f.tell(),curblocksize)
            found = False
    
        if found:
            blocksize = curblocksize
            offset = f.tell()
        else:
            f.seek(curblocksize, os.SEEK_CUR)
            blocksize_check = (np.fromfile(f,dtype=np.int32,count=1))[0]
            if swap: blocksize_check = blocksize_check.byteswap()
            if (curblocksize != blocksize_check):
                print("something wrong")
                sys.exit()
            curblock_num += 1
      
    f.close()
      
    if ((not found) and (not only_list_blocks)):
        print("Error: block not found")
        sys.exit()
    
    if (not only_list_blocks):
        return offset,blocksize
 
# ----- read data block -----
 
def read_block(filename, block, parttype=-1, physical_velocities=True, arepo=0, no_masses=False, verbose=False):
    if (verbose):
	    print("reading block", block)
  
    blockadd=0
    blocksub=0
  
    if arepo==0:
        if (verbose):	
	        print("Gadget format")
        blockadd=0
    if arepo==1:
        if (verbose):	
	        print("Arepo format")
        blockadd=1	
    if arepo==2:
        if (verbose):
	        print("Arepo extended format")
        blockadd=4	
    if no_masses==True:
        if (verbose):	
	        print("No mass block present")    
        blocksub=1
		 
    if parttype not in [-1,0,1,2,3,4,5]:
        print("wrong parttype given")
        sys.exit()
  
    if os.path.exists(filename):
        curfilename = filename
    elif os.path.exists(filename+".0"):
        curfilename = filename+".0"
    else:
        print("file not found:", filename)
        print("and:", curfilename)
        sys.exit()
  
    head = snapshot_header(curfilename)
    format = head.format
    swap = head.swap
    npart = head.npart
    massarr = head.massarr
    nall = head.nall
    filenum = head.filenum
    redshift = head.redshift
    time = head.time
    del head
  
  # - description of data blocks -
  # add or change blocks as needed for your Gadget version
    data_for_type = np.zeros(6,bool) # should be set to "True" below for the species for which data is stored in the data block
    dt = np.float32 # data type of the data in the block
    if block=="POS ":
        data_for_type[:] = True
        dt = np.dtype((np.float32,3))
        block_num = 2
    elif block=="VEL ":
        data_for_type[:] = True
        dt = np.dtype((np.float32,3))
        block_num = 3
    elif block=="ID  ":
        data_for_type[:] = True
        dt = np.uint32
        block_num = 4
    elif block=="MASS":
        data_for_type[np.where(massarr==0)] = True
        block_num = 5
        if parttype>=0 and massarr[parttype]>0:   
            if (verbose):	
	            print("filling masses according to massarr")   
            return np.ones(nall[parttype],dtype=dt)*massarr[parttype]
    elif block=="U   ":
        data_for_type[:] = True
        dt = np.dtype((np.float32))
        block_num = 6#-blocksub
    elif block=="RHO ":
        data_for_type[0] = True
        block_num = 7-blocksub
    elif block=="VOL ":
        data_for_type[0] = True
        block_num = 8-blocksub 
    elif block=="CMCE":
        data_for_type[0] = True
        dt = np.dtype((np.float32,3))
        block_num = 9-blocksub 
    elif block=="AREA":
        data_for_type[0] = True
        block_num = 10-blocksub
    elif block=="NFAC":
        data_for_type[0] = True
        dt = np.dtype(np.int32)	
        block_num = 11-blocksub
    elif block=="NE  ":
        data_for_type[0] = True
        block_num = 8+blockadd-blocksub
    elif block=="NH  ":
        data_for_type[0] = True
        block_num = 9+blockadd-blocksub
    elif block=="HSML":
        data_for_type[0] = True
        block_num = 10+blockadd-blocksub
    elif block=="SFR ":
        data_for_type[0] = True
        block_num = 11+blockadd-blocksub
    elif block=="AGE ":
        data_for_type[4] = True
        block_num = 12+blockadd-blocksub
    elif block=="Z   ":
        data_for_type[0] = True
        data_for_type[4] = True
        block_num = 13+blockadd-blocksub
    elif block=="BHMA":
        data_for_type[5] = True
        block_num = 14+blockadd-blocksub
    elif block=="BHMD":
        data_for_type[5] = True
        block_num = 15+blockadd-blocksub
    elif block=="COOR":
        data_for_type[0] = True
        block_num = -1 
    else:
        print("Sorry! Block type", block, "not known!")
        sys.exit()
  # - end of block description -

    if (block_num < 0 and format==1):
        print("Sorry! Block number of", block, "not known! Unable to read this block from format 1 file!")
        sys.exit() 
    
    actual_data_for_type = np.copy(data_for_type)  
    if parttype >= 0:
        actual_data_for_type[:] = False
        actual_data_for_type[parttype] = True
        if data_for_type[parttype]==False:
            print("Error: no data for specified particle type", parttype, "in the block", block)   
            sys.exit()
    elif block=="MASS":
        actual_data_for_type[:] = True  
    
    allpartnum = np.int64(0)
    species_offset = np.zeros(6,np.int64)
    for j in range(6):
        species_offset[j] = allpartnum
        if actual_data_for_type[j]:
            allpartnum += nall[j]
    filenum=1  
    for i in range(filenum): # main loop over files
        if filenum>1:
            curfilename = filename+"."+str(i)
      
        if i>0:
            head = snapshot_header(curfilename)
            npart = head.npart  
            del head
      
        curpartnum = np.int32(0)
        cur_species_offset = np.zeros(6,np.int64)
        for j in range(6):
            cur_species_offset[j] = curpartnum
            if data_for_type[j]:
                curpartnum += npart[j]
    
        if parttype>=0:
            actual_curpartnum = npart[parttype]      
            add_offset = cur_species_offset[parttype] 
        else:
            actual_curpartnum = curpartnum
            add_offset = np.int32(0)
      
        offset,blocksize = find_block(curfilename,format,swap,block,block_num)
    
        if i==0: # fix data type for ID if long IDs are used
            if block=="ID  ":
                if blocksize == np.dtype(dt).itemsize*curpartnum * 2:
                    dt = np.uint64 
        
        if np.dtype(dt).itemsize*curpartnum != blocksize:
            print("something wrong with blocksize! expected =",np.dtype(dt).itemsize*curpartnum,"actual =",blocksize)
            sys.exit()
    
        f = open(curfilename,'rb')
        f.seek(offset + add_offset*np.dtype(dt).itemsize, os.SEEK_CUR)  
        curdat = np.fromfile(f,dtype=dt,count=actual_curpartnum) # read data
        f.close()  
        if swap:
            curdat.byteswap(True)  
      
        if i==0:
            data = np.empty(allpartnum,dt)
    
        for j in range(6):
            if actual_data_for_type[j]:
                if block=="MASS" and massarr[j]>0: # add mass block for particles for which the mass is specified in the snapshot header
                    data[species_offset[j]:species_offset[j]+npart[j]] = massarr[j]
                else:
                    if parttype>=0:
                        data[species_offset[j]:species_offset[j]+npart[j]] = curdat
                    else:
                        data[species_offset[j]:species_offset[j]+npart[j]] = curdat[cur_species_offset[j]:cur_species_offset[j]+npart[j]]
                species_offset[j] += npart[j]

        del curdat

    if physical_velocities and block=="VEL " and redshift!=0:
        data *= math.sqrt(time)

    return data
  
# ----- list all data blocks in a format 2 snapshot file -----

def list_format2_blocks(filename):
    if (not os.path.exists(filename)):
        print("file not found:", filename)
        sys.exit()
  
    head = snapshot_header(filename)
    format = head.format
    swap = head.swap
    del head
  
    if (format != 2):
        print("not a format 2 snapshot file")
        sys.exit()
            
    print("#   BLOCK   OFFSET   SIZE")
    print("-------------------------")
  
    find_block(filename, format, swap, "XXXX", 0, only_list_blocks=True)
  
    print("-------------------------")


In [2]:
from tqdm import tqdm
from joblib import Parallel, delayed

In [3]:
## This function takes the vector product between axis and the                                                                                                                 
## normalized vector in the z direction to be the axis of rotation                                                                                                             
## Then it rotates the body by the angle between the axis and the z vector                                                                                                     

def rotation_axis_angle(axis):
    #function rotation_axis_angle, axis

    unitz=[0.0,0.0,1.0]

    angle=np.pi+(-1.0)*np.arccos(axis[2]/np.sqrt(axis[0]**2+axis[1]**2+axis[2]**2))

    ##print,"length axis rotation axis angle,", sqrt(axis[0]^2+axis[1]^2+axis[2]^2)                                                                                                

    ## u= z vector product axis                                                                                                                                                    

    u = np.zeros(3)

    ##print, "axis length", sqrt(axis[0]^2+axis[1]^2+axis[2]^2)                                                                                                                    

    u[0]=-unitz[2]*axis[1]

    u[1]=unitz[2]*axis[0]

    u[2]=0.0

    n=np.sqrt(u[0]**2+u[1]**2)

    u=u/n

    ##kronecker tensor                                                                                                                                                             

    kronecker=np.zeros(shape=(3,3))

    kronecker[0,0]=1.0

    kronecker[0,1]=0.0

    kronecker[0,2]=0.0

    kronecker[1,0]=0.0

    kronecker[1,1]=1.0

    kronecker[1,2]=0.0

    kronecker[2,0]=0.0

    kronecker[2,1]=0.0

    kronecker[2,2]=1.0

    ucrossu=np.zeros(shape=(3,3))

    for i in range (0,3) :
        for j in range (0,3):
            ucrossu[i,j]=u[i]*u[j]

    u_x=np.zeros(shape=(3,3))

    u_x[0,0]=0.0

    u_x[0,1]=(-1.0)*u[2]

    u_x[0,2]=u[1]

    u_x[1,0]=u[2]

    u_x[1,1]=0.0

    u_x[1,2]=(-1.0)*u[0]

    u_x[2,0]=(-1.0)*u[1]

    u_x[2,1]=u[0]

    u_x[2,2]=0.0

    rotation=np.zeros(shape=(3,3))

    rotation = ucrossu + np.cos(angle)*(kronecker-ucrossu)+ np.sin(angle)*u_x

    return rotation

In [4]:
def allign_disk(pos, vel, mass, ids):
    rr=np.sqrt(pos[0,:]**2 + pos[1,:]**2 + pos[2,:]**2)#distance of stars                                                                                                     
    Jdisk = np.zeros((3, len(pos[0,:])))
    
    Jdisk[0,:] = pos[1,:]*vel[2,:]-pos[2,:]*vel[1,:]

    Jdisk[1,:] = pos[2,:]*vel[0,:]-pos[0,:]*vel[2,:]

    Jdisk[2,:] = pos[0,:]*vel[1,:]-pos[1,:]*vel[0,:]

    rcyl=30.0                        #choose 30 kpc to select particles, 4 kpc turned out not to be sufficient enough                                                               

    wr4kpc= np.where((rr < rcyl))


    jtot=np.zeros(3)

    jtot[0]=np.sum(Jdisk[0,wr4kpc])

    jtot[1]=np.sum(Jdisk[1,wr4kpc])

    jtot[2]=np.sum(Jdisk[2,wr4kpc])

    magjtot=np.sqrt(jtot[0]**2+jtot[1]**2+jtot[2]**2)

    jtot_normalised=jtot/magjtot

    rot_align_J=rotation_axis_angle(jtot_normalised)


    #pp=np.matmul(rot_align_J,pp)

    #vv=np.matmul(rot_align_J,vv)

    pos=np.matmul(rot_align_J,pos)

    vel=np.matmul(rot_align_J,vel)




    rdisk=np.sqrt(pos[0,:]**2+pos[1,:]**2)

    zshrink=0.7

    zstart=10.0

    wr4kpc=np.where((rdisk < rcyl) & (pos[2,:] < zstart))

    x=pos[0:3,wr4kpc]
               
    v=vel[0:3,wr4kpc]
    
    m = mass[wr4kpc]
        
    ids = ids[wr4kpc]

    min_particles=1000
                   
    while len(wr4kpc) > min_particles:

        jtot[0]=np.sum(x[1,:]*v[2,:]-x[2,:]*v[1,:])

        jtot[1]=np.sum(x[2,:]*v[0,:]-x[0,:]*v[2,:])

        jtot[2]=np.sum(x[0,:]*v[1,:]-x[1,:]*v[0,:])

        magjtot=np.sqrt(jtot[0]^2+jtot[1]^2+jtot[2]^2)

        jtot_normalised=jtot/magjtot

        rot_align_J=rotation_axis_angle(jtot_normalised)

        #pp=np.matmul(rot_align_J, pp)

        #vv=np.matmul(rot_align_J,vv)

        pos=np.matmul(rot_align_J,pos)

        vel=np.matmul(rot_align_J,vel)

        rdisk=np.sqrt(pos[0,:]**2+pos[1,:]**2)

        zstart=zstart*zshrink

        wr4kpc=np.where((rdisk < rcyl) & (np.abs(pos[2,:]) < zstart))

        x=pos[0:3,wr4kpc]

        v=vel[0:3,wr4kpc]

        m = mass[wr4kpc]
        
        ids = ids[wr4kpc]

    return x, v, m, ids

In [5]:
#function for reading in snaps and fixing the mass to solar masses
def read_snap(gal):
    dm_mass =read_block('/mnt/d/Snaps/snap_'+gal,"MASS",parttype=1)
    dm_pos =read_block('/mnt/d/Snaps/snap_'+gal,"POS ",parttype=1)
    dm_vel =read_block('/mnt/d/Snaps/snap_' +gal,"VEL ",parttype=1)

    disk_mass =read_block('/mnt/d/Snaps/snap_'+gal,"MASS",parttype=2)
    disk_pos =read_block('/mnt/d/Snaps/snap_'+gal,"POS ",parttype=2)
    disk_vel =read_block('/mnt/d/Snaps/snap_'+gal,"VEL ",parttype=2)

    bulg_mass =read_block('/mnt/d/Snaps/snap_'+gal,"MASS",parttype=3)
    bulg_pos =read_block('/mnt/d/Snaps/snap_'+gal,"POS ",parttype=3)
    bulg_vel =read_block('/mnt/d/Snaps/snap_'+gal,"VEL ",parttype=3)
    
    id_disk = read_block('/mnt/d/Snaps/snap_'+gal, "ID  ", parttype=2)
    
    dm_mass = dm_mass * 10000000000
    disk_mass = disk_mass * 10000000000
    bulg_mass = bulg_mass * 10000000000
    return dm_mass, dm_pos, dm_vel, disk_mass, disk_pos, disk_vel, bulg_mass, bulg_pos, bulg_vel, id_disk

In [6]:
#algorithm for centering positions of stars and also alligning the Lz vector with the Z-direction
def center_positions(dm_mass, dm_pos, dm_vel, disk_mass, disk_pos, disk_vel, bulg_mass, bulg_pos, bulg_vel, id_disk):
    X_dm = dm_pos[:,0]
    Y_dm = dm_pos[:,1]
    Z_dm = dm_pos[:,2]

    X_disk = disk_pos[:,0]
    Y_disk = disk_pos[:,1]
    Z_disk = disk_pos[:,2]

    X_bulg = bulg_pos[:,0]
    Y_bulg = bulg_pos[:,1]
    Z_bulg = bulg_pos[:,2]

    U_dm = dm_vel[:,0]
    V_dm = dm_vel[:,1]
    W_dm = dm_vel[:,2]

    U_disk = disk_vel[:,0]
    V_disk = disk_vel[:,1]
    W_disk = disk_vel[:,2]

    U_bulg = bulg_vel[:,0]
    V_bulg = bulg_vel[:,1]
    W_bulg = bulg_vel[:,2]
    
    meanx = np.mean(X_bulg)
    meany = np.mean(Y_bulg)
    meanz = np.mean(Z_bulg)
    meanu = np.mean(U_bulg)
    meanv = np.mean(V_bulg)
    meanw = np.mean(W_bulg)
    meanxd = np.mean(X_dm)
    meanyd = np.mean(Y_dm)
    meanzd = np.mean(Z_dm)
    meanud = np.mean(U_dm)
    meanvd = np.mean(V_dm)
    meanwd = np.mean(W_dm)

    X_bulg = X_bulg - meanx
    Y_bulg = Y_bulg - meany
    Z_bulg = Z_bulg - meanz
    X_disk = X_disk - meanx
    Y_disk = Y_disk - meany
    Z_disk = Z_disk - meanz
    U_disk = U_disk - meanu
    V_disk = V_disk - meanv
    W_disk = W_disk - meanw
    U_bulg = U_bulg - meanu
    V_bulg = V_bulg - meanv
    W_bulg = W_bulg - meanw
    X_dm = X_dm - meanxd
    Y_dm = Y_dm - meanyd
    Z_dm = Z_dm - meanzd
    U_dm = U_dm - meanud
    V_dm = V_dm - meanvd
    W_dm = W_dm - meanwd
    
    mass = disk_mass
    ids = id_disk
    posdisk = np.vstack((X_disk,Y_disk,Z_disk))
    veldisk = np.vstack((U_disk,V_disk,W_disk))
    x, v, m, ids = allign_disk(posdisk, veldisk, mass, ids)
    
    X_disk = x[0,:]
    Y_disk = x[1,:]
    Z_disk = x[2,:]
    U_disk = v[0,:]
    V_disk = v[1,:]
    W_disk = v[2,:]
    
    #Centering Disk and Bulge
    for i in (400,300,200,100,50,25,1,0.01,0.001):
        rho = X_bulg**2 + Y_bulg**2 + Z_bulg**2
        ichoose = np.where((rho<i))
        meanx = np.mean(X_bulg[ichoose])
        meany = np.mean(Y_bulg[ichoose])
        meanz = np.mean(Z_bulg[ichoose])
        meanu = np.mean(U_bulg[ichoose])
        meanv = np.mean(V_bulg[ichoose])
        meanw = np.mean(W_bulg[ichoose])

        X_bulg = X_bulg - meanx
        Y_bulg = Y_bulg - meany
        Z_bulg = Z_bulg - meanz
        X_disk = X_disk - meanx
        Y_disk = Y_disk - meany
        Z_disk = Z_disk - meanz
        U_disk = U_disk - meanu
        V_disk = V_disk - meanv
        W_disk = W_disk - meanw
        U_bulg = U_bulg - meanu
        V_bulg = V_bulg - meanv
        W_bulg = W_bulg - meanw
        
        posdisk = np.vstack((X_disk,Y_disk,Z_disk))
        veldisk = np.vstack((U_disk,V_disk,W_disk))
        x, v, m, ids = allign_disk(posdisk, veldisk, m, ids)
        X_disk = x[0,:]
        Y_disk = x[1,:]
        Z_disk = x[2,:]
        U_disk = v[0,:]
        V_disk = v[1,:]
        W_disk = v[2,:]
        
    #Centering Dark Matter Halo
    for i in (4000000, 1000000,500000, 250000, 100000,50000, 10000,1000, 100, 10, 1):
        rho = X_dm**2 + Y_dm**2 + Z_dm**2
        ichoose = np.where((rho<i))
        meanxd = np.mean(X_dm[ichoose])
        meanyd = np.mean(Y_dm[ichoose])
        meanzd = np.mean(Z_dm[ichoose])
        meanud = np.mean(U_dm[ichoose])
        meanvd = np.mean(V_dm[ichoose])
        meanwd = np.mean(W_dm[ichoose])
    
        X_dm = X_dm - meanxd
        Y_dm = Y_dm - meanyd
        Z_dm = Z_dm - meanzd
        U_dm = U_dm - meanud
        V_dm = V_dm - meanvd
        W_dm = W_dm - meanwd
        
        disk_mass = m
        id_disk = ids
    #returning them all (maybe too long)
    return X_bulg, Y_bulg, Z_bulg, X_disk, Y_disk, Z_disk, X_dm, Y_dm, Z_dm, U_bulg, V_bulg, W_bulg, U_disk, V_disk, W_disk, U_dm, V_dm, W_dm, disk_mass, bulg_mass, dm_mass, id_disk

In [7]:
#Algorithm that puts all the functions together and saves the phasespace information
def main(gal):
    dm_m, dm_p, dm_v, disk_m, disk_p, disk_v, bulg_m, bulg_p, bulg_v, id_d = read_snap(gal)
    
    X_bul, Y_bul, Z_bul, X_dis, Y_dis, Z_dis, X_d, Y_d, Z_d, U_bul, V_bul, W_bul, U_dis, V_dis, W_dis, U_d, V_d, W_d,disk_mass,bulg_mass,dm_mass, id_disk = center_positions(dm_m, dm_p, dm_v, disk_m, disk_p, disk_v, bulg_m, bulg_p, bulg_v, id_d)

    disk_pos = np.zeros((len(X_dis[0]),3))
    disk_pos[:,0] = X_dis[0]
    disk_pos[:,1] = Y_dis[0]
    disk_pos[:,2] = Z_dis[0]

    bulg_pos = np.zeros((len(X_bul),3))
    bulg_pos[:,0] = X_bul
    bulg_pos[:,1] = Y_bul
    bulg_pos[:,2] = Z_bul

    dm_pos = np.zeros((len(X_d),3))
    dm_pos[:,0] = X_d
    dm_pos[:,1] = Y_d
    dm_pos[:,2] = Z_d

    disk_vel = np.zeros((len(U_dis[0]),3))
    disk_vel[:,0] = U_dis[0]
    disk_vel[:,1] = V_dis[0]
    disk_vel[:,2] = W_dis[0]

    bulg_vel = np.zeros((len(U_bul),3))
    bulg_vel[:,0] = U_bul
    bulg_vel[:,1] = V_bul
    bulg_vel[:,2] = W_bul

    dm_vel = np.zeros((len(U_d),3))
    dm_vel[:,0] = U_d
    dm_vel[:,1] = V_d
    dm_vel[:,2] = W_d
    
    #Saving stuff. Maybe saving the mass is redundant 
    np.save('/mnt/d/npy3/disk_pos_'+gal+'.npy', disk_pos)
    np.save('/mnt/d/npy3/bulg_pos_'+gal+'.npy', bulg_pos)
    np.save('/mnt/d/npy3/dm_pos_'+gal+'.npy', dm_pos)
    np.save('/mnt/d/npy3/disk_vel_'+gal+'.npy', disk_vel)
    np.save('/mnt/d/npy3/bulg_vel_'+gal+'.npy', bulg_vel)
    np.save('/mnt/d/npy3/dm_vel_'+gal+'.npy', dm_vel)
    np.save('/mnt/d/npy3/id_disk_'+gal+'.npy', id_disk)
    np.save('/mnt/d/npy3/bulg_mass_'+gal+'.npy', bulg_mass)
    np.save('/mnt/d/npy3/disk_mass_'+gal+'.npy', disk_mass)
    np.save('/mnt/d/npy3/dm_mass_'+gal+'.npy', dm_mass)

In [8]:
#all snaps
glist = ['010','020','030','040','050','060','070','080','090','100','110','120','130','140','150','160','170','180','190','600','605','606','607','608','609','610','611','612','613','614','615','620','625','630','635','640','645','650','655','660','665','670','675','680','685','690','695']

In [9]:
for gal in glist:
    main(gal)